# Movie Hybrid Recommendation Logic
This notebook demonstrates a hybrid movie recommendation system combining **content-based filtering** and **collaborative filtering** (using SVD). We use the MovieLens dataset (movies.csv and ratings.csv) to build and explain the logic.

---

## Step 1: Import Libraries and Load Dataset

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import numpy as np

# Load datasets
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# Display sample data
print('Movies DataFrame:')
print(movies.head())

print('\nRatings DataFrame:')
print(ratings.head())

## Step 2: Content-Based Filtering
We will use TF-IDF vectorization on movie genres to find movies similar to a given movie based on genre similarity.

In [ ]:

# Create TF-IDF matrix from genres
tfidf = TfidfVectorizer(token_pattern='[a-zA-Z0-9-]+')
tfidf_matrix = tfidf.fit_transform(movies['Genres'])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Map movie titles to indices for quick lookup
indices = pd.Series(movies.index, index=movies['Title']).drop_duplicates()

def content_recommendations(title, cosine_sim=cosine_sim):
    """Return top 10 movies similar to the given movie title based on genres."""
    idx = indices.get(title, None)
    if idx is None:
        return "Movie not found in database."
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Exclude itself
    movie_indices = [i[0] for i in sim_scores]
    return movies['Title'].iloc[movie_indices]

# Example usage
content_recommendations('Toy Story (1995)')

## Step 3: Collaborative Filtering using SVD
We use matrix factorization (SVD) on the user-item rating matrix to predict missing ratings and recommend movies.

In [ ]:

# Create user-item rating matrix
user_movie_ratings = ratings.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)

# Perform SVD
U, sigma, Vt = svds(user_movie_ratings, k=50)
sigma = np.diag(sigma)

# Predict ratings
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns=user_movie_ratings.columns, index=user_movie_ratings.index)

def collaborative_recommendations(user_id, preds_df=preds_df, movies_df=movies, original_ratings=ratings, num_recommendations=10):
    """Recommend movies for a user based on predicted ratings."""
    user_row_number = user_id - 1  # UserID starts at 1
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get user's rated movies
    user_data = original_ratings[original_ratings.UserID == user_id]
    user_full = user_data.merge(movies_df, how='left', left_on='MovieID', right_on='MovieID').sort_values(['Rating'], ascending=False)
    
    # Recommend movies the user hasn't rated yet
    recommendations = movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])]
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                                            left_on='MovieID',
                                            right_on='MovieID')
    recommendations = recommendations.rename(columns={user_row_number: 'PredictedRating'})
    recommendations = recommendations.sort_values('PredictedRating', ascending=False)
    
    return recommendations[['Title', 'Genres']].head(num_recommendations)

# Example usage
collaborative_recommendations(1, preds_df, movies, ratings, 5)

## Step 4: Hybrid Recommendation
Combine content-based and collaborative filtering results to give better personalized recommendations.

In [ ]:

def hybrid_recommendations(user_id, favorite_movie_title, top_n=5):
    # Get collaborative recommendations for user
    collab_recs = collaborative_recommendations(user_id, preds_df, movies, ratings, num_recommendations=20)
    
    # Get content based recommendations for user's favorite movie
    content_recs = content_recommendations(favorite_movie_title)
    
    if isinstance(content_recs, str):  # If movie not found
        return content_recs
    
    # Find intersection of recommended movie titles
    hybrid_recs = collab_recs[collab_recs['Title'].isin(content_recs)].head(top_n)
    
    # If intersection is small, append more from collaborative recommendations
    if len(hybrid_recs) < top_n:
        additional = collab_recs[~collab_recs['Title'].isin(hybrid_recs['Title'])].head(top_n - len(hybrid_recs))
        hybrid_recs = pd.concat([hybrid_recs, additional])
    
    return hybrid_recs.reset_index(drop=True)

# Example usage
hybrid_recommendations(1, 'Toy Story (1995)', 5)


### Conclusion
This notebook demonstrated a hybrid movie recommendation system combining content-based filtering (using genres) and collaborative filtering (using SVD). This approach balances personalized user preferences with movie attributes for better recommendations.